# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
#gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
opened_df = pd.read_csv('output_data/weather_data.csv', index_col ='Unnamed: 0')
opened_df


,city,lat,long,temp,humidity,cloudiness,windspeed,max temp
0,Minas,-34.38,-55.24,8.93,70,75,6.20,9.00
1,Rikitea,-23.12,-134.97,20.70,69,28,10.31,20.70
2,Mayo,38.89,-76.51,24.72,78,1,1.50,26.11
3,Severo-Kuril'sk,50.68,156.12,8.41,90,100,2.47,8.41
4,Rabo de Peixe,37.80,-25.58,18.18,77,40,3.10,18.33
...,...,...,...,...,...,...,...,...
539,Nacala,-14.54,40.67,18.16,91,5,3.99,18.16
540,Ati,13.22,18.34,29.73,46,19,4.87,29.73
541,Kingsport,36.55,-82.56,25.48,74,75,2.10,27.22
542,Sibu,2.30,111.82,25.00,94,40,3.10,25.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
opened_df.dropna()

# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = opened_df[["lat", "long"]]

# set humidity values
humidity = opened_df["humidity"].astype(float)




In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
# narrow by: temp between 70 and 80, windspeed less than 5, 0 cloudiness
# create new df, drop rest of rows
vacay_df = opened_df.loc[(opened_df["temp"] > 20) & (opened_df["temp"] < 25) & (opened_df["windspeed"] < 5) & (opened_df["cloudiness"] == 0)]
#df2 = df.loc[((df['a'] > 1) & (df['b'] > 0)) | ((df['a'] < 1) & (df['c'] == 100))]

vacay_df

,city,lat,long,temp,humidity,cloudiness,windspeed,max temp
147,Svobodnyy,51.40,128.13,22.38,55,0,4.20,22.38
289,Nísia Floresta,-6.09,-35.21,24.00,88,0,2.60,24.00
304,Ajdabiya,30.76,20.23,22.13,86,0,4.40,22.13
308,Rāvar,31.27,56.81,23.09,20,0,0.21,23.09
355,Erzin,36.96,36.20,24.44,88,0,0.59,24.44
387,Santa Marinella,42.03,11.85,21.67,69,0,2.78,21.67
426,São José da Coroa Grande,-8.90,-35.15,23.41,80,0,2.35,23.41
437,Mizdah,31.45,12.98,23.35,40,0,1.56,23.35
500,Canals,38.96,-0.58,24.39,84,0,0.45,25.56
512,Chunskiy,56.08,99.63,23.94,58,0,1.62,23.94


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# hotel_df = vacay_df
#hotel_df["Hotel Name"] = ""

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
